In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, LogisticRegression

In [2]:
n_actions = 10
n_features = 20
n_data_list = [250, 500, 1000, 2000, 4000, 8000]

In [51]:
n_data = n_data_list[1]

x = np.random.normal(size=(n_data, n_features))
Wx0 = np.random.normal(size=(n_actions, n_features))
Bx0 = np.random.normal(size=(n_actions, 1))

Wx = np.random.normal(size=(n_actions, n_features))
Bx = np.random.normal(size=(n_actions, 1))

Wxa_x = np.random.normal(size=(n_features,))
Wxa_a = np.random.normal(size=(n_actions,))

def sigmoid(x):
    return 1./(1.+np.exp(x))

def feature2action0(x):
    prob_a = sigmoid(Wx0@x.T+Bx0).T
    prob_a = prob_a/prob_a.sum(axis=1, keepdims=True)
    return prob_a

def feature2action(x):
    prob_a = sigmoid(Wx@x.T+Bx).T
    prob_a = prob_a/prob_a.sum(axis=1, keepdims=True)
    return prob_a

def sample_actions(prob_a):
    data_size, action_size = prob_a.shape
    actions = np.array([np.random.choice(action_size, p=prob_a[i]) for i in range(data_size)])
    result = np.zeros_like(prob_a)
    result[np.arange(data_size), actions] = 1
    return result, actions

def action2reward(x, a):
    return (Wxa_x@x.T+Wxa_a@a.T).T

In [52]:
prob_a0 = feature2action0(x)
a0, actions0 = sample_actions(prob_a0)
r0 = action2reward(x, a0)

prob_a = feature2action(x)
a, actions = sample_actions(prob_a)
r = action2reward(x, a)

In [59]:
model = LogisticRegression(multi_class='multinomial', max_iter=200)
model.fit(X=x, y=actions0)
a_pred0 = model.predict_proba(x)

model = LinearRegression()
model.fit(X=np.concatenate([x, a_pred0], axis=1), y=r0)

r_pred = model.predict(X=np.concatenate([x, prob_a], axis=1))

In [63]:
(((r-r_pred)/r)**2).mean()

42.66458776552922